<a href="https://colab.research.google.com/github/djangointhehills/Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [122]:
import numpy as np
import math
import copy
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron

# Perceptron

In [11]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [189]:
class Perceptron_Unit:
    def __init__(self, learning_rate, activation_f, max_iterations, weights):
        self.weights = [0] * weights
        self.activation_f = activation_f
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations

    def update_weights(self, x, y_hat, y):
        for i in range(len(self.weights)):
            self.weights[i] += self.learning_rate * (y - y_hat) * x[i]

    def train(self, X, Y):
        # Apend 1 at beginning of each instacne of accomodate bias.
        X = np.c_[ np.ones(len(X)),X] 

        # Initialise number of weights and add bias.
        

        epochs = 0
        train = True
        while train and epochs < self.max_iterations:
          epochs += 1
          train = False

          # Train over all the X data instances.
          for i in range(len(X)):
              y_hat = self.activation_f(np.dot(self.weights, X[i]))
              if y_hat != Y[i]:
                  train = True
                  self.update_weights(X[i], y_hat, Y[i])
        print('Training complete!')

    def update(self, x,y):
        pred = self.activation_f(np.dot(self.weights, x))
        if pred != y:
          self.update_weights(x, pred, y)

    def predict(self, x):

        prediction = self.activation_f(np.dot(self.weights, x))
        return prediction
    
    def get_acuracy(self, preds,y):
        no_correct = 0
        for i in range(len(preds)):
          if preds[i] == y[i]:
            no_correct += 1
        accuracy = no_correct / len(preds)
        return accuracy

    def predict_score(self, X,Y):
        predictions = self.predict(X)
        accuracy = self.get_acuracy(predictions,Y)
        print("Accuracy =", accuracy)
    
    def get_weights(self):
      return self.weights



In [247]:
class Perceptron:
  def __init__(self, learning_rate, activation_f, max_iterations):
    self.perceptrons = {}
    self.activation_f = activation_f
    self.learning_rate = learning_rate
    self.max_iterations = max_iterations

  def train(self, X, Y):
    # Get the number of weights needed.
    num_weights = 1 + len(X[0])

    # Create perceptron for each label.
    labels = np.unique(Y)
    if len(labels) < 3:
      self.perceptrons[labels[0]] = Perceptron_Unit(self.learning_rate, self.activation_f, self.max_iterations, num_weights)
    else:
      for label in labels:
        self.perceptrons[label] = Perceptron_Unit(self.learning_rate, self.activation_f, self.max_iterations, num_weights)

    for i in range(len(X)):
      # Add bias term
      instance = np.insert(X[i], 0, 1)

      prediction = (-math.inf, None)

      # Get max probability label.
      for label in self.perceptrons: 
        prob = self.perceptrons[label].predict(instance)
        if Y[i] == label:
          
            if prob > prediction[0]:
              prediction = (prob, label)
        else:
          if prob > prediction[0]:
            prediction = (prob, label)
 
      
      # Check if prediction was correct. If not, update.
      if prediction[1] != Y[i]:
        for label in self.perceptrons:
          if Y[i] == label:
            self.perceptrons[label].update(instance, 1)
          else:
            self.perceptrons[label].update(instance, 0)

  
  def get_weights(self,):
    weights = {}
    for unit in self.perceptrons:
      weights[unit] = self.perceptrons[unit].get_weights()
    
    return weights
    
    # predict(self, X):

      

        




  


In [248]:
model = Perceptron(.3, sigmoid, 10)

In [249]:
model.train(X_train, y_train)

In [250]:
model.get_weights()

{0: [0.34230571220578593,
  0.3905751209542286,
  1.7252023905350347,
  -2.9446702153800026,
  -1.2263763748915084],
 1: [0.3879711174378214,
  0.040199837906757185,
  -0.8587854860684593,
  -2.4012087928785064,
  -2.312060407445914],
 2: [-0.9431122710662532,
  -1.532187090584522,
  -1.9441822016194301,
  5.004791505097575,
  3.3189281337984533]}

# Test

In [51]:
model = Perceptron(.3, sigmoid)

In [52]:
X = [[1,2,3,4],
 [3,2,4,4],
 [1,2,12,3]]

Y = [0,0,1]



In [64]:
model.train(X,Y)

Training complete!


In [17]:
predictions = model.predict(X)

In [18]:
print(predictions)

[0.0006024306711822066, 0.0002846986661957373, 0.9981355218907151]


# Experiementation with Iris Data Set

## Load Data

In [20]:
iris = datasets.load_iris()

In [25]:
x_data = iris['data']
y_data = iris['target']

In [121]:
x_data.shape

(150, 4)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=42)

## Create Model

In [111]:
model = Perceptron(.1, sigmoid, classifier=True)

In [130]:
model.train(X_train, y_train)

Training complete!


In [131]:
model.predict_score(X_test, y_test)

Accuracy = 0.3


In [133]:
predictions = model.predict(X_test)
print(predictions)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [119]:
weights = model.get_weights()
weights

array([-0.19080644, -0.44389125, -0.38161288,  0.66930572, -0.91525548])

# Compare with Sklearn Percetron

In [123]:
sk_model = Perceptron()

In [126]:
sk_model.fit(X_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [127]:
sk_model.score(X_test, y_test)

0.72

In [129]:
sk_predictions = sk_model.predict(X_test)
print(sk_predictions)

[0 0 2 0 1 0 0 2 2 0 2 0 0 0 0 0 2 0 2 2 0 2 0 2 2 2 2 2 0 0 0 0 0 0 0 2 0
 0 0 0 2 0 0 0 0 0 2 2 0 2]
